# Evaluation of Dash
## Imports

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.style.use('seaborn-v0_8-whitegrid')

import tol_colors as tc
plt.rc('axes', prop_cycle=plt.cycler('color', list(tc.tol_cset('bright'))))
#plt.cm.register_cmap('iridescent', tc.tol_cmap('iridescent'))
plt.rc('image', cmap='iridescent')

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 12,
    "font.size": 11, # same size as ticks label
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 11,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1,
    "axes.titlesize": 12
}

plt.rcParams.update(tex_fonts)

## Utils

In [2]:
def get_latest_file(path, suffix):
    paths = sorted(Path(path).iterdir(), key=os.path.getmtime, reverse=True)
    paths = [str(p) for p in paths]
    for p in paths:
        if p.endswith(suffix):
            return p

def relative_std(x):
  # https://github.com/pandas-dev/pandas/issues/33517
  if not isinstance(x, pd.Series):
        raise TypeError
  
  return np.nanstd(x)/np.nanmean(x)

## Micro-Benchmarks
### GPU / CPU Speedup
#### Dense

In [ ]:
path_dense = get_latest_file("micro_benchmarks/data", "_dense.csv")
print(path_dense)
dense_data = pd.read_csv(path_dense, skipinitialspace=True)
dense_data = dense_data.drop(columns=["crt_base_size", "run", "q_acc", "gpu_mem_usage"])
dense_data = dense_data.groupby(["dimensions", "type"]).aggregate(['mean', 'std', relative_std])
dense_data = dense_data.unstack()
dense_data

In [ ]:
dense_speedup = dense_data["runtime"]["mean"]["CPU"]/dense_data["runtime"]["mean"]["GPU"]
dense_speedup

#### Conv2d

In [ ]:
path_conv2d = get_latest_file("micro_benchmarks/data", "_conv2d.csv")
print(path_conv2d)
conv2d_data = pd.read_csv(path_conv2d, skipinitialspace=True)
conv2d_data = conv2d_data.drop(columns=["crt_base_size", "run", "q_acc"])
conv2d_data = conv2d_data.groupby(["dimensions", "type"]).aggregate(['mean', 'std', relative_std])
conv2d_data = conv2d_data.unstack()
conv2d_data

In [ ]:
conv2d_speedup = conv2d_data["runtime"]["mean"]["CPU"]/conv2d_data["runtime"]["mean"]["GPU"]
conv2d_speedup

#### Approx. ReLU

In [ ]:
path_approx_relu = get_latest_file("micro_benchmarks/data", "_approx_relu.csv")
print(path_approx_relu)

relu_data = pd.read_csv(path_approx_relu, skipinitialspace=True)
relu_data = relu_data.drop(columns=["run", "q_acc", "crt_base_size"])
relu_data = relu_data.groupby(["dimensions", "type", "relu_acc"]).aggregate(['mean', 'std', relative_std])
relu_data = relu_data.unstack([-2, -1])
relu_data

In [ ]:
relu_speedup = relu_data["runtime"]["mean"]["CPU"]/relu_data["runtime"]["mean"]["GPU"]
relu_speedup

#### Approx. Sign Activation

In [ ]:
path_approx_sign = get_latest_file("micro_benchmarks/data", "_sign_activation.csv")
print(path_approx_sign)


sign_data = pd.read_csv(path_approx_sign, skipinitialspace=True)
sign_data = sign_data.drop(columns=["run", "q_acc", "crt_base_size"])
sign_data = sign_data.groupby(["dimensions", "type", "sign_acc"]).aggregate(['mean', 'std', relative_std])
sign_data = sign_data.unstack([-2, -1])
sign_data

In [ ]:
sign_speedup = sign_data["runtime"]["mean"]["CPU"]/sign_data["runtime"]["mean"]["GPU"]
sign_speedup

#### Rescaling

In [ ]:
rescaling = get_latest_file("micro_benchmarks/data", "_rescaling.csv")
print(rescaling)


rescaling_data = pd.read_csv(rescaling, skipinitialspace=True)
rescaling_data = rescaling_data.drop(columns=["run", "crt_base_size"])
rescaling_data = rescaling_data.groupby(["dimensions", "type"]).aggregate(['mean', 'std', relative_std])
rescaling_data = rescaling_data.unstack([-1])
rescaling_data

In [ ]:
rescaling_speedup = rescaling_data["runtime"]["mean"]["CPU"]/rescaling_data["runtime"]["mean"]["GPU"]
rescaling_speedup

#### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 1.5), nrows=1, ncols=5)
# spcaing
fig.subplots_adjust(left=0, bottom=None, right=None, top=None, wspace=0.15, hspace=None)

# relu
relu_speedup.plot(kind="bar", ax=ax[0])
ax[0].set_title("ReLU")
ax[0].set_xlabel("Dimension (power of 2)")
ax[0].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[0].set_ylabel("Speedup")
ax[0].get_legend().remove()

# sign
sign_speedup.plot(kind="bar", ax=ax[1])
ax[1].set_title("Sign")
ax[1].set_xlabel("Dimension (power of 2)")
ax[1].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[1].get_legend().remove()

# dense
dense_speedup.plot(kind="bar", ax=ax[2])
ax[2].set_title("Dense")
ax[2].set_xlabel("Dimension")
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation = 0)

# conv2d
conv2d_speedup.plot(kind="bar", ax=ax[3])
ax[3].set_title("Conv2D")
ax[3].set_xlabel("Dimension")
ax[3].set_xticklabels(['64x64x3', '128x128x3', '256x256x3'], rotation = 0)

# rescaling
rescaling_speedup.plot(kind="bar", ax=ax[4])
ax[4].set_title("Scaling")
ax[4].set_xlabel("Dimension (power of 2)")
ax[4].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[4].set_xticklabels(ax[4].get_xticklabels(), rotation = 0)

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, ["99.0\%", "100.0\% Sign-Gadget Accuracy"], loc='lower center', bbox_to_anchor=(0.45, -0.4, 0, 0), ncols=2)

fig.savefig('micro_benchmarks/data/all_speedup.pdf', format='pdf', bbox_inches='tight')

### GPU Memory Usage

##### Dense

In [ ]:
path_dense = get_latest_file("micro_benchmarks/data", "_dense.csv")
print(path_dense)
dense_data = pd.read_csv(path_dense, skipinitialspace=True)
dense_data = dense_data[(dense_data["type"] == "GPU") & (dense_data["run"] >= 5)]
dense_data = dense_data.drop(columns=["crt_base_size", "run", "q_acc", "runtime", "type"])
dense_data["gpu_mem_usage"] /= (1024*1024)
dense_data = dense_data.groupby(["dimensions"]).aggregate(['mean', 'std', relative_std])
dense_data

#### Conv2d

In [ ]:
path_conv2d = get_latest_file("micro_benchmarks/data", "_conv2d.csv")
print(path_conv2d)
conv2d_data = pd.read_csv(path_conv2d, skipinitialspace=True)
conv2d_data = conv2d_data[(conv2d_data["type"] == "GPU") & (conv2d_data["run"] >= 5)]
conv2d_data = conv2d_data.drop(columns=["crt_base_size", "run", "q_acc", "runtime", "type"])
conv2d_data["gpu_mem_usage"] /= (1024*1024)
conv2d_data = conv2d_data.groupby(["dimensions"]).aggregate(['mean', 'std', relative_std])
conv2d_data

#### Approx. ReLU

In [ ]:
path_approx_relu = get_latest_file("micro_benchmarks/data", "_approx_relu.csv")
print(path_approx_relu)

relu_data = pd.read_csv(path_approx_relu, skipinitialspace=True)
relu_data = relu_data[(relu_data["type"] == "GPU") & (relu_data["run"] >= 5)]
relu_data = relu_data.drop(columns=["run", "q_acc", "crt_base_size", "runtime", "type"])
relu_data["gpu_mem_usage"] /= (1024*1024)
relu_data = relu_data.groupby(["dimensions", "relu_acc"]).aggregate(['mean', 'std', relative_std])
relu_data = relu_data.unstack()
relu_data

In [ ]:
# improvement factor 
factor = relu_data["gpu_mem_usage"]["mean"]
factor[100.0] / factor[99.0]

#### Approx. Sign

In [ ]:
path_approx_sign = get_latest_file("micro_benchmarks/data", "_sign_activation.csv")
print(path_approx_sign)

approx_sign_data = pd.read_csv(path_approx_sign, skipinitialspace=True)
approx_sign_data = approx_sign_data[(approx_sign_data["type"] == "GPU") & (approx_sign_data["run"] >= 5)]
approx_sign_data = approx_sign_data.drop(columns=["run", "q_acc", "crt_base_size", "runtime", "type"])
approx_sign_data["gpu_mem_usage"] /= (1024*1024)
approx_sign_data = approx_sign_data.groupby(["dimensions", "sign_acc"]).aggregate(['mean', 'std', relative_std])
approx_sign_data = approx_sign_data.unstack()
approx_sign_data

In [ ]:
# improvement factor 
factor = approx_sign_data["gpu_mem_usage"]["mean"]
factor[100.0] / factor[99.0]

#### Rescaling

In [ ]:
rescaling = get_latest_file("micro_benchmarks/data", "_rescaling.csv")
print(rescaling)

rescaling_data = pd.read_csv(rescaling, skipinitialspace=True)
rescaling_data = rescaling_data[(rescaling_data["type"] == "GPU") & (rescaling_data["run"] >= 5)]
rescaling_data = rescaling_data.drop(columns=["run", "crt_base_size", "runtime", "type"])
rescaling_data["gpu_mem_usage"] /= (1024*1024)
rescaling_data = rescaling_data.groupby(["dimensions"]).aggregate(['mean', 'std', relative_std])
rescaling_data

#### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 1.5), nrows=1, ncols=5)
# spcaing
fig.subplots_adjust(left=0, bottom=None, right=None, top=None, wspace=0.2, hspace=None)

relu_data["gpu_mem_usage"]["mean"].plot(kind="bar", ax=ax[0], label="ReLU")
ax[0].set_title("ReLU")
ax[0].set_xlabel("Dimension (power of 2)")
ax[0].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[0].set_ylabel("Memory Usage (MB)")
ax[0].get_legend().remove()

approx_sign_data["gpu_mem_usage"]["mean"].plot(kind="bar", ax=ax[1], label="Sign")
ax[1].set_title("Sign")
ax[1].set_xlabel("Dimension (power of 2)")
ax[1].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[1].set_yticks([0, 250, 500, 750, 1000])
ax[1].get_legend().remove()

dense_data["gpu_mem_usage"]["mean"].plot(kind="bar", ax=ax[2], label="Dense")
ax[2].set_title("Dense")
ax[2].set_xlabel("Dimension")
ax[2].set_yticks([0, 10, 20, 30, 40])
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation = 0)

conv2d_data["gpu_mem_usage"]["mean"].plot(kind="bar", ax=ax[3], label="Conv2D")
ax[3].set_title("Conv2D")
ax[3].set_xlabel("Dimension")
ax[3].set_xticklabels(['64x64x3', '128x128x3', '256x256x3'], rotation = 0)

rescaling_data["gpu_mem_usage"]["mean"].plot(kind="bar", ax=ax[4], label="Scaling")
ax[4].set_title("Scaling")
ax[4].set_xlabel("Dimension (power of 2)")
ax[4].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[4].set_xticklabels(ax[4].get_xticklabels(), rotation = 0)
ax[4].set_yticks([0, 200, 400, 600, 800])

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, ["99.0\%", "100.0\% Sign-Gadget Accuracy"], loc='lower center', bbox_to_anchor=(0.45, -0.4, 0, 0), ncols=2)

fig.savefig('micro_benchmarks/data/gpu_memory.pdf', format='pdf', bbox_inches='tight')

### CPU-Scalability
#### Approx. ReLU

In [ ]:
path_relu_scale = get_latest_file("micro_benchmarks/data/scalability", "_approx_relu.csv")
print(path_relu_scale)

relu_scale_data = pd.read_csv(path_relu_scale, skipinitialspace=True)
relu_scale_data = relu_scale_data.drop(columns=["run", "q_acc", "crt_base_size", "relu_acc", "type"])
relu_scale_data = relu_scale_data.groupby(["dims", "nr_threads"]).aggregate(['mean', 'std', relative_std])
relu_scale_data = relu_scale_data.unstack([-2])
relu_scale_data

#### Sign Activation

In [ ]:
path_sign_scale = get_latest_file("micro_benchmarks/data/scalability", "_sign_activation.csv")
print(path_sign_scale)

sign_scale_data = pd.read_csv(path_sign_scale, skipinitialspace=True)
sign_scale_data = sign_scale_data.drop(columns=["run", "crt_base_size", "type"])
sign_scale_data = sign_scale_data.groupby(["dims", "nr_threads"]).aggregate(['mean', 'std', relative_std])
sign_scale_data = sign_scale_data.unstack([-2])
sign_scale_data

#### Dense

In [ ]:
path_dense_scale = get_latest_file("micro_benchmarks/data/scalability", "_dense.csv")
print(path_dense_scale)

dense_scale_data = pd.read_csv(path_dense_scale, skipinitialspace=True)
dense_scale_data = dense_scale_data.drop(columns=["run", "q_acc", "crt_base_size", "type"])
dense_scale_data = dense_scale_data.groupby(["dims", "nr_threads"]).aggregate(['mean', 'std', relative_std])
dense_scale_data = dense_scale_data.unstack([-2])
dense_scale_data

#### Conv2d

In [ ]:
path_conv2d_scale = get_latest_file("micro_benchmarks/data/scalability", "_conv2d.csv")
print(path_conv2d_scale)

conv2d_scale_data = pd.read_csv(path_conv2d_scale, skipinitialspace=True)
conv2d_scale_data = conv2d_scale_data.drop(columns=["run", "q_acc", "crt_base_size", "type"])
conv2d_scale_data = conv2d_scale_data.groupby(["dims", "nr_threads"]).aggregate(['mean', 'std', relative_std])
conv2d_scale_data = conv2d_scale_data.unstack([-2])
conv2d_scale_data

#### Rescaling

In [ ]:
path_rescaling_scale = get_latest_file("micro_benchmarks/data/scalability", "_rescaling.csv")
print(path_rescaling_scale)

rescaling_scale_data = pd.read_csv(path_rescaling_scale, skipinitialspace=True)
rescaling_scale_data = rescaling_scale_data.drop(columns=["run", "crt_base_size", "type"])
rescaling_scale_data = rescaling_scale_data.groupby(["dims", "nr_threads"]).aggregate(['mean', 'std', relative_std])
rescaling_scale_data = rescaling_scale_data.unstack([-2])
rescaling_scale_data

#### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12.5, 1.5), nrows=1, ncols=5)
# spcaing
fig.subplots_adjust(left=0, bottom=None, right=None, top=None, wspace=0.15, hspace=None)

# relu
relu_scale_data["runtime"]["mean"].plot(logy=True, ax=ax[0], marker='x')
ax[0].set_title("ReLU")
ax[0].set_xlabel("Number of Threads")
ax[0].xaxis.set_ticks(np.arange(2, 18, 2))
ax[0].set_ylabel("Runtime (ms)")
ax[0].legend(loc="lower left", bbox_to_anchor=(0,-0.65), ncols=3, handletextpad=0.4, columnspacing=0.4, handlelength=1)

# sign
sign_scale_data["runtime"]["mean"].plot(logy=True, ax=ax[1], marker='x')
ax[1].set_title("Sign")
ax[1].set_xlabel("Number of Threads")
ax[1].xaxis.set_ticks(np.arange(2, 18, 2))
ax[1].legend(loc="lower left", bbox_to_anchor=(0,-0.65), ncols=3, handletextpad=0.4, columnspacing=0.4, handlelength=1)

# dense
dense_scale_data["runtime"]["mean"].plot(logy=True, ax=ax[2], marker='x')
ax[2].set_title("Dense")
ax[2].set_xlabel("Number of Threads")
ax[2].xaxis.set_ticks(np.arange(2, 18, 2))
ax[2].legend(loc="lower left", bbox_to_anchor=(0.05,-0.65), ncols=3, handletextpad=0.4, columnspacing=0.4, handlelength=1)

# conv
conv2d_scale_data["runtime"]["mean"].plot(logy=True, ax=ax[3], marker='x')
ax[3].set_title("Conv2D")
ax[3].set_xlabel("Number of Threads")
ax[3].xaxis.set_ticks(np.arange(2, 18, 2))
ax[3].legend(labels=['64x64x3', '128x128x3', '256x256x3'], loc="lower left", bbox_to_anchor=(-0.15,-0.65), ncols=3, handletextpad=0.4, columnspacing=0.4, handlelength=0.5)

# rescale
sign_scale_data["runtime"]["mean"].plot(logy=True, ax=ax[4], marker='x')
ax[4].set_title("Scaling")
ax[4].set_xlabel("Number of Threads")
ax[4].xaxis.set_ticks(np.arange(2, 18, 2))
ax[4].legend(loc="lower left", bbox_to_anchor=(0,-0.65), ncols=3, handletextpad=0.4, columnspacing=0.4, handlelength=1)

fig.savefig('micro_benchmarks/data/scalability.pdf', format='pdf', bbox_inches='tight')

### Merged Plots

In [ ]:
fig, ax = plt.subplots(figsize=(12, 1.5), nrows=1, ncols=5)
# spcaing
fig.subplots_adjust(left=0, bottom=None, right=None, top=None, wspace=0.15, hspace=None)

# relu
relu_speedup.plot(kind="bar", ax=ax[0])
ax[0].set_title("ReLU")
ax[0].set_xlabel("Dimension (power of 2)")
ax[0].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[0].set_ylabel("Speedup")
ax[0].get_legend().remove()

# sign
sign_speedup.plot(kind="bar", ax=ax[1])
ax[1].set_title("Sign")
ax[1].set_xlabel("Dimension (power of 2)")
ax[1].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[1].get_legend().remove()

# dense
dense_speedup.plot(kind="bar", ax=ax[2])
ax[2].set_title("Dense")
ax[2].set_xlabel("Dimension")
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation = 0)

# conv2d
conv2d_speedup.plot(kind="bar", ax=ax[3])
ax[3].set_title("Conv2D")
ax[3].set_xlabel("Dimension")
ax[3].set_xticklabels(['64x64x3', '128x128x3', '256x256x3'], rotation = 0)

# rescaling
rescaling_speedup.plot(kind="bar", ax=ax[4])
ax[4].set_title("Scaling")
ax[4].set_xlabel("Dimension (power of 2)")
ax[4].set_xticklabels([7, 8, 9, 10, 11, 12, 13, 14], rotation = 0)
ax[4].set_xticklabels(ax[4].get_xticklabels(), rotation = 0)

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, ["99.0\%", "100.0\% Sign-Gadget Accuracy"], loc='lower center', bbox_to_anchor=(0.45, -0.4, 0, 0), ncols=2)

fig.savefig('micro_benchmarks/data/all_speedup.pdf', format='pdf', bbox_inches='tight')

## Model Benchmarks

In [ ]:
#path_models = get_latest_file("model_benchmarks/data", "_plain_models.csv")
path_models = get_latest_file("model_benchmarks/data", "_sgx_models.csv")
print(path_models)
models_data = pd.read_csv(path_models, skipinitialspace=True)
models_data = models_data[models_data["relu_acc"] == 100]
models_data = models_data.drop(columns=["target_crt_base_size", "label", "infered_label", "relu_acc"])
models_data = models_data.groupby(["model", "type"]).aggregate(['mean', 'std', relative_std])
models_data = models_data.unstack()
models_data

### Add available data of other frameworks

In [ ]:
# Reset the index so that we can iterate through the numbers.
# This will help us to get the x tick positions
df = models_data["runtime"]["mean"]
df = df.reset_index()
# Add data from gnn paper
df["GNNP"] = [60, 1520, 210, 3340, 7, 3, 97000, np.nan]
df["SecureML"] = [180, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
df["MiniONN"] = [140, 400, np.nan, 5740, np.nan, np.nan, np.nan, 72000]
df["GAZELLE"] = [30, 30, 50, 329, np.nan, np.nan, np.nan, 3560]
df["CryptoNets"] = [np.nan, 297500, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
df["DeepSecure"] = [np.nan, np.nan, 9670, np.nan, np.nan, np.nan, np.nan, np.nan]
df["ExPC"] = [np.nan, np.nan, np.nan, 5100, np.nan, np.nan, np.nan, np.nan]
df["FHE-DiNN"] = [np.nan, np.nan, np.nan, np.nan, 400, 1650, np.nan, np.nan]
df["MUSE/SIMC"] = [np.nan, np.nan, np.nan, 800, np.nan, np.nan, np.nan, 7860]
df["model"] = df["model"].apply(lambda x: x.removeprefix("MODEL_"))
df

In [ ]:
# Remove realy old and slow frameworks from the comparison
df = df.drop(columns = ["FHE-DiNN", "ExPC"])
df = df[(df.model != "E_100") & (df.model != "E_30")]
df

### Compute communication overhead in theoretical model

In [ ]:
# Communication Overhead in Dash compressed labels

input_size_mnist = 28 * 28 * 1
input_size_cifar10 = 32 * 32 * 3

def compute_comm_overhead_size(input_size, crt_base_size):
    overhead = 1
    effective_transfer_speed = 1 * pow(10, 9) # 1 Gigabits
    plain_datatype = 16 # bit
    quantized_datatype = 64 # bit
    output_size = 10 # nr classes
    compressed_label_size = 128 # bit
    
    garbled_input_size = input_size * crt_base_size * compressed_label_size
    garbled_output_size = output_size * crt_base_size * compressed_label_size
    communication_size = input_size * plain_datatype # communicate plain input
    communication_size = communication_size + garbled_input_size
    communication_size = communication_size + garbled_output_size
    communication_size = communication_size + output_size * quantized_datatype
    mb = communication_size / 8 / 1024 / 1024
    return mb

comm_overhead_size = dict()
comm_overhead_size["A"] = compute_comm_overhead_size(input_size_mnist, 8)
comm_overhead_size["B"] = compute_comm_overhead_size(input_size_mnist, 9)
comm_overhead_size["C"] = compute_comm_overhead_size(input_size_mnist, 9)
comm_overhead_size["D"] = compute_comm_overhead_size(input_size_mnist, 8)
comm_overhead_size["F_GNNP_POOL_REPL"] = compute_comm_overhead_size(input_size_cifar10, 7)
comm_overhead_size["F_MINIONN_POOL_REPL"] = compute_comm_overhead_size(input_size_cifar10, 7)

print("Communication Overhead (Size):")
for key, value in comm_overhead_size.items():
    print(f"MODEL-{key}: {value} mb")

def compute_comm_overhead(input_size, crt_base_size):
    overhead = 1
    effective_transfer_speed = 1 * pow(10, 9) # 1 Gigabits
    plain_datatype = 16 # bit
    quantized_datatype = 64 # bit
    output_size = 10 # nr classes
    compressed_label_size = 128 # bit
    
    garbled_input_size = input_size * crt_base_size * compressed_label_size
    garbled_output_size = output_size * crt_base_size * compressed_label_size
    communication_size = input_size * plain_datatype # communicate plain input
    communication_size = communication_size + garbled_input_size
    communication_size = communication_size + garbled_output_size
    communication_size = communication_size + output_size * quantized_datatype
    transfer_time = communication_size / effective_transfer_speed
    milliseconds = transfer_time * 1000
    milliseconds = milliseconds + milliseconds * overhead
    return milliseconds
    
comm_overhead = dict()
comm_overhead["A"] = compute_comm_overhead(input_size_mnist, 8)
comm_overhead["B"] = compute_comm_overhead(input_size_mnist, 9)
comm_overhead["C"] = compute_comm_overhead(input_size_mnist, 9)
comm_overhead["D"] = compute_comm_overhead(input_size_mnist, 8)
comm_overhead["F_GNNP_POOL_REPL"] = compute_comm_overhead(input_size_cifar10, 7)
comm_overhead["F_MINIONN_POOL_REPL"] = compute_comm_overhead(input_size_cifar10, 7)

print("Communication Overhead (Time):")
for key, value in comm_overhead.items():
    print(f"MODEL-{key}: {value} ms")

### Distribution of computation and communication over whole runtime in ms

In [ ]:
# Plot runtime divided into compute- and communication-time
tmp = df[df.model != "F_GNNP_POOL_REPL"]
tmp = tmp[tmp.model != "F_MINIONN_POOL_REPL"]

fig, ax = plt.subplots(figsize=(3.5,0.7))
ax.barh(tmp.model, tmp.GPU + list(comm_overhead.values())[:-2], label="Communication")
ax.barh(tmp.model, tmp.GPU, label="Computation")
ax.set_xlabel("Runtime (ms)")
ax.set_ylabel("Model")
ax.legend(loc='upper left', bbox_to_anchor=(-0.1, -0.5, 0, 0), ncols=2)
fig.savefig('model_benchmarks/data/distribution_communication_computation.pdf', format='pdf', bbox_inches='tight')

### Distribution of computation and communication over whole runtime in percent

In [ ]:
list(comm_overhead.values()) / (df["GPU"] + list(comm_overhead.values()))

### Comparison of Dash's model runtimes against other frameworks

In [ ]:
# Add communication overhead to computation runtime
comm_overhead = list(comm_overhead.values())
df.GPU = df.GPU + comm_overhead
df.CPU = df.CPU + comm_overhead
df

In [ ]:
# Plot Runtimes
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
hatches = ['','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']
#color_hatch_combinations = [(c, h) for h in hatches for c in colors]
color_hatch_combinations = [(colors[i % (len(colors)-1)], h) for i, h in enumerate(hatches)]

# See also https://stackoverflow.com/questions/53399022/pandas-plot-bar-without-nan-values-spaces
fig, ax = plt.subplots(figsize=(12.5, 1.5))
ax.set_yscale('log')
ax.set_yticks([10, 100, 1000, 10000, 100000])

# Width of bars
width = 1

# Create emptly lists for x tick positions and names
x_ticks, x_ticks_pos = [], []

# Counter for helping with x tick positions
count = 0

my_labels = []

# Go through each row of the dataframe
for idx, row in df.iterrows():
    # This will be the first bar position for this row
    count += 1
    
    model = row["model"]

    sorted_row = row[1:].copy()
    sorted_row = sorted_row.sort_values()

    # This will be the start of the first bar for this row
    start_idx = count - width / 2
    # This will be the end of the last bar for this row
    end_idx = start_idx
    # For each column in the wanted columns,
    # if the row is not null,
    # add the bar to the plot
    # Also update the end position of the bars for this row
    # for column_idx, column in enumerate(df.drop(["model"], axis=1).columns):
    #     # This checks if the row had any not NULL value in the desired columns
    #     # in other words, it checks if there was any bar for this row
    #     if row[column] == row[column]:
    #         ax.bar(count, row[column], color=colors[column_idx], width=width, label=column)
    #         count += 1
    #         end_idx += width

    for row_index, v in enumerate(sorted_row.index):
        if sorted_row[v] == sorted_row[v]:
            c, h = color_hatch_combinations[row.index.get_loc(v)-1]
            ax.bar(count, sorted_row.iloc[row_index], color=c, hatch=h, width=width, label=v)
            if v not in my_labels:
                my_labels.append(v)
            count += 1
            end_idx += width

    # This checks if the row had any not NULL value in the desired columns
    # in other words, it checks if there was any bar for this row
    # if yes, add the center of all the row's bars and the row's name (A,B,C) to the respective lists
    if end_idx != start_idx:
        x_ticks_pos.append((end_idx + start_idx) / 2)
        x_ticks.append(model)

# Now set the x_ticks
ax.set_xticks(x_ticks_pos, x_ticks)
ax.set_xticklabels(["MODEL-A", "MODEL-B", "MODEL-C", "MODEL-D", "MODEL-F-SMALL", "MODEL-F"])
ax.set_ylabel("Runtime (ms)")

# Modify my_labels
for idx, l in enumerate(my_labels):
    if l == "CPU":
        my_labels[idx] = "Dash (CPU)"
    elif l == "GPU":
        my_labels[idx] = "Dash (GPU)"

# Also plot the legends
# and make sure to not display duplicate labels
# The below code is taken from:
# https://stackoverflow.com/questions/13588920/stop-matplotlib-repeating-labels-in-legend
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), my_labels, loc='lower center', bbox_to_anchor=(0.5, -0.7, 0, 0), ncols=5)
fig.savefig('model_benchmarks/data/models_runtime_comparison.pdf', format='pdf', bbox_inches='tight')

In [ ]:
print(colors)

### Runtime Distribution over Layer-Types

In [ ]:
path_models = get_latest_file("model_benchmarks/data", "_runtime_distribution_evaluation.csv")
print(path_models)
runtime_dist_data = pd.read_csv(path_models, skipinitialspace=True)
runtime_dist_data[runtime_dist_data["relu_acc"] == 100]

runtime_dist_data = runtime_dist_data.drop(columns=["target_crt_base_size", "relu_acc"])
runtime_dist_data = runtime_dist_data.groupby(["model", "type", "layer"]).aggregate(['mean', 'std', relative_std])

# add 0 second entries for missing layers (not all models contain all layers)
runtime_dist_data = runtime_dist_data.unstack(0)["runtime"]["mean"]
runtime_dist_data.fillna(0, inplace=True)
runtime_dist_data

In [ ]:
raw_data_cpu = dict()
raw_data_gpu = dict()
for model in runtime_dist_data.columns:
    s = runtime_dist_data[model]["CPU"].sum()
    for layer in runtime_dist_data[model]["CPU"].index:
        if layer not in raw_data_cpu:
            raw_data_cpu[layer] = []
        raw_data_cpu[layer].append(runtime_dist_data[model]["CPU"][layer] / s * 100)

for model in runtime_dist_data.columns:
    s = runtime_dist_data[model]["GPU"].sum()
    for layer in runtime_dist_data[model]["GPU"].index:
        if layer not in raw_data_gpu:
            raw_data_gpu[layer] = []
        raw_data_gpu[layer].append(runtime_dist_data[model]["GPU"][layer] / s * 100)

model_names = [
    "A",
    "B",
    "C",
    "D",
    "E-100",
    "E-30",
    "F-SMALL",
    "F",
]

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
hatches = ['','//', '\\\\', '||', '--', '++', 'xx', 'oo', 'OO', '..', '**']
#color_hatch_combinations = [(c, h) for h in hatches for c in colors]
color_hatch_combinations = [(colors[i % (len(colors)-1)], h) for i, h in enumerate(hatches)]

fig, ax = plt.subplots(figsize=(10, 2.7), nrows=1, ncols=2, sharey=True)
fig.tight_layout(pad=2.8)
left_cpu = len(runtime_dist_data.columns)*[0]
left_gpu = len(runtime_dist_data.columns)*[0]
color_cnt = 0
for idx, layer in enumerate(raw_data_cpu):
    c, h = color_hatch_combinations[idx]
    ax[0].barh(model_names, raw_data_cpu[layer], left=left_cpu, label=layer, color=c, hatch=h)
    left_cpu = [x + y for x, y in zip(left_cpu, raw_data_cpu[layer])]
    ax[1].barh(model_names, raw_data_gpu[layer], left=left_gpu, label=layer, color=c, hatch=h)
    left_gpu = [x + y for x, y in zip(left_gpu, raw_data_gpu[layer])]

ax[0].set_title("CPU Evaluation")
ax[0].set_xlabel("Runtime in \%")
ax[1].set_title("GPU Evaluation")
ax[1].set_xlabel("Runtime in \%")
ax[0].set_ylabel("Model")

handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
layer = ["ReLU", "Conv2D", "Dense", "Rescale", "Sign"]
ax[0].legend(
    by_label.values(),
    layer,
    loc="lower center",
    bbox_to_anchor=(1.1, -0.5, 0, 0),
    ncol=5,
)
fig.savefig('model_benchmarks/data/runtime_distribution_layer.pdf', format='pdf', bbox_inches='tight')
